## Build the trigger

The Production Centre uses Docker images and deploys them in a Cloud based environment for large scale production campaigns.

This step will trigger the build of a Docker image containing the trigger. 

We will invoke components of the Ellip Platform Continuous Integration.

The input for trigering the build of Docker image containing the data transformation application uses the pom, an XML file that contains information about the data transformation application and configuration details used by Maven to build the Docker image.

The goal of this Jupyter Notebook is to build of the data transformation application Docker image.

* Define the data transformation application information

In [5]:
trigger = dict([('artifact_id', 'tg-ecopotential-00003'),
                ('community', 'ec-ecopotential')])

* Set the authentication information:

In [7]:
import getpass 

username = raw_input("What is your Jenkins username? ")
api_key = getpass.getpass('And your Jenkins API key:')

What is your Jenkins username? fbrito
And your Jenkins API key:········


* First the import of the required Python libraries:

In [8]:
import jenkins
import time
import lxml.etree as etree
import sys
import requests
import os
import string
import hashlib
import urllib2
import pytz
from datetime import datetime
import time
import owslib
from owslib.wps import monitorExecution
import uuid
from owslib.wps import WebProcessingService


* Create two simple functions to submit the build request and to monitor it:

In [9]:
def submit_build_job(server, job, sleepSecs=3):
    
    last_build = server.get_job_info(job)['lastCompletedBuild']['number']
    
    server.build_job(job, {'RELEASE': 'true'}) 
    
    while server.get_job_info(job)['lastBuild']['number'] == last_build:
        
        time.sleep(sleepSecs)
        
    return server.get_job_info(job)['lastBuild']['number']

In [10]:
def monitor_build_job(server, job, sleepSecs=15):
    
    last_build_number = server.get_job_info(job)['lastBuild']['number']
    
    while server.get_build_info(job, last_build_number)['building']:
        
        time.sleep(sleepSecs)
    
    if str(server.get_build_info(job, last_build_number)['result']) == 'SUCCESS':
        
        return True
    
    if str(server.get_build_info(job, last_build_number)['result']) == 'FAILURE':
        
        return False

* Establish a connection with the build component of the Ellip Platform Continuous Integration:

In [11]:
server = jenkins.Jenkins(url='https://build.terradue.com',
                        username=username,
                        password=api_key)

* Create the build request path:

In [12]:
job = 'triggers/%s/%s/docker' % (trigger['community'], 
                                      trigger['artifact_id'])

In [13]:
print(job)

triggers/ec-ecopotential/tg-ecopotential-00003/docker


* Submit the build job:

In [14]:
submit_build_job(server, job)

92

* Monitor the build request:

In [15]:
if monitor_build_job(server, job):
    print 'Job build completed'
else:
    print 'Job build failed'

Job build completed
